# Grove Motor Driver v1.3  
http://wiki.seeedstudio.com/Grove-I2C_Motor_Driver_V1.3/

In [1]:
from meerkat.base import time
from meerkat import motor

#### Stop Command for Debugging  
This cell is useful when prototyping.  If a command throws an error before setting both motor speeds to 0, the board will remain on and possibly overheat.

In [2]:
m = motor.GroveMotor(bus_n=1)
m.stop()
print("Stop Done")

Stop Done


#### I2C Bus Address
Initialize the motor controller class.  Default I2C address is `0x0F` which the four switches are set to `1111` - another option is to use Python's binary representation `0b00001111`

In [3]:
hex(0b00001111)

'0xf'

#### Initialize Driver Class with Default I2C Address

In [4]:
m = motor.GroveMotor(bus_n=1)

#### Motor Metadata

In [5]:
m.csv_writer.device

{'name': 'Grove Motor Driver',
 'version_hw': '1.3',
 'version_sw': '1.0',
 'accuracy': None,
 'precision': '1 microstep',
 'bus': '<meerkat.i2c_pi.WrapI2C object at 0xb33f44f0>',
 'state': None,
 'active': None,
 'error': None,
 'dtype': None,
 'description': 'I2C DC and stepper motor controller',
 'urls': 'http://wiki.seeedstudio.com/Grove-I2C_Motor_Driver_V1.3/',
 'manufacturer': 'Seeed Studio',
 'calibration_date': None}

#### Motor Status Header

In [6]:
m.csv_writer.header

['description',
 'freq',
 'm1_speed',
 'm2_speed',
 'm_direction',
 'mode',
 'phase',
 'steps']

#### Motor Status

In [7]:
m.get("test_1a_init")

['test_1a_init', 31372, 0, 0, '', 'dc', 2, 0]

To be clear, the headers names aligned with the data above are:  

| description | freq | m1_speed | m2_speed | m_direction | mode | phase | steps |
| ----------- | ---- | -------- | -------- | ----------- | ---- | ----- | ----- |
| test_1_init | 31372 | 0 | 0 |  | dc | 2 | 0 |

Begin saving motor state after each configuration change

In [8]:
m.write("test_1b_init")

## Stepper Motor  

#### Micro Steps

In [9]:
m.step_micro(1001, delay=0.0001, verbose=False)

time.sleep(1)

m.step_micro(-1001, delay=0.0001, verbose=False)

In [10]:
m.get("test_2a_microsteps")

['test_2a_microsteps', 31372, 0, 0, 'step_ccw', 'micro_step', 2, -1001]

In [11]:
m.write("test_2b_microsteps")

Output JSON of the current driver state

In [12]:
m.publish("test_2c_microsteps")

'{"description": "test_2c_microsteps", "freq": 31372, "m1_speed": 0, "m2_speed": 0, "m_direction": "step_ccw", "mode": "micro_step", "phase": 2, "steps": -1001, "std_time_ms": "2020-01-07 07:44:44.789433"}'

#### Full Steps

In [13]:
m.step_full(100, delay=0.0001, verbose=False)

time.sleep(1)

m.step_full(-100, delay=0.0001, verbose=False)

In [14]:
m.get("test_3a_full_steps")

['test_3a_full_steps', 31372, 0, 0, 'step_ccw', 'full_step', 2, -100]

In [15]:
m.write("test_3b_full_steps")

In [16]:
m.publish("test_3c_full_steps")

'{"description": "test_3c_full_steps", "freq": 31372, "m1_speed": 0, "m2_speed": 0, "m_direction": "step_ccw", "mode": "full_step", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:44:57.053332"}'

## DC Motor  
For this portion, disconnect the stepper motor from pinout J3 and connect a DC motor to J1.  

Note: The Grove Arduino driver uses -100 to 100 for accepted motor speed values, however their driver converts it to -255 to 255 because that is the board's accepted input values.  -255 to 255 is used here so that finer step control is possible.  Bench test speeds needed for a motor, if 100 is used with this Python driver, some motors may not start!

### LED Colors, Directions Codes and Descriptions

| M1 | M2 | Code | Description |
| -- | -- | ---- | ----------- |
| <span style='color:green'>green</span> | <span style='color:green'>green</span> | cw         | both clockwise         |
| <span style='color:red'>red</span>     | <span style='color:red'>red</span>     | ccw        | both counter clockwise |
| <span style='color:green'>green</span> | <span style='color:green'>green</span> | m1m2cw     | both clockwise         |
| <span style='color:red'>red</span>     | <span style='color:red'>red</span>     | m1m2cc     | both counter clockwise |
| <span style='color:green'>green</span> | <span style='color:red'>red</span>     | m1cw_m2ccw | M1 clockwise, M2 counter clockwise |
| <span style='color:red'>red</span>     | <span style='color:green'>green</span> | m1ccw_m2cw | M1 counter clockwise, M2 clockwise |

In [17]:
# ensure stop condition before switching to dc motor control
m.stop()

In [18]:
m.set_mode(mode_type="dc")
m.set_frequency(f_Hz=31372)

#### DC Motor Clockwise and Counter Clockwise

In [19]:
for d in ["cw", "ccw"]:
    m.set_direction(d)
    m.set_speed(motor_id=1, motor_speed=255)
    print(m.get("test_5a_dc_{}".format(d)))
    m.write("test_5b_dc_{}".format(d))
    print(m.publish("test_5c_dc_{}".format(d)))
    time.sleep(2)

    m.stop(motor_id=1)
    print(m.get("test_5d_dc_{}".format(d)))
    m.write("test_5e_dc_{}".format(d))
    print(m.publish("test_5f_dc_{}".format(d)))

['test_5a_dc_cw', 1, 255, 0, 10, 'dc', 2, -100]
{"description": "test_5c_dc_cw", "freq": 1, "m1_speed": 255, "m2_speed": 0, "m_direction": 10, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:45:57.727118"}
['test_5d_dc_cw', 1, 0, 0, 10, 'dc', 2, -100]
{"description": "test_5f_dc_cw", "freq": 1, "m1_speed": 0, "m2_speed": 0, "m_direction": 10, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:45:59.733225"}
['test_5a_dc_ccw', 1, 255, 0, 5, 'dc', 2, -100]
{"description": "test_5c_dc_ccw", "freq": 1, "m1_speed": 255, "m2_speed": 0, "m_direction": 5, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:45:59.736868"}
['test_5d_dc_ccw', 1, 0, 0, 5, 'dc', 2, -100]
{"description": "test_5f_dc_ccw", "freq": 1, "m1_speed": 0, "m2_speed": 0, "m_direction": 5, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:46:01.742649"}


Ramp up the motor speed while publishing JSON and recording to CSV

In [20]:
for speed in range(0, 255, 25):
    m.set_speed(motor_id=1, motor_speed=speed)
    print("> State after motor speed set to {}".format(speed))
    desc = "test_6_{}".format(speed)
    m.write(description=desc)
    print("JSON Format:", m.publish(description=desc))
    time.sleep(0.25)
m.stop()

> State after motor speed set to 0
JSON Format: {"description": "test_6_0", "freq": 1, "m1_speed": 0, "m2_speed": 0, "m_direction": 5, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:46:09.635459"}
> State after motor speed set to 25
JSON Format: {"description": "test_6_25", "freq": 1, "m1_speed": 25, "m2_speed": 0, "m_direction": 5, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:46:09.889894"}
> State after motor speed set to 50
JSON Format: {"description": "test_6_50", "freq": 1, "m1_speed": 50, "m2_speed": 0, "m_direction": 5, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:46:10.142815"}
> State after motor speed set to 75
JSON Format: {"description": null, "freq": 1, "m1_speed": 75, "m2_speed": 0, "m_direction": 5, "mode": "dc", "phase": 2, "steps": -100, "std_time_ms": "2020-01-07 07:46:10.396324", "name": "Grove Motor Driver", "title": null, "format": null, "encoding": "utf-8", "bytes": null, "hash": null, "sc

Open the CSV batch file and parse the metadata column data

In [21]:
m.csv_writer.path

'2020_01_07_07_44_33_data.csv'

In [22]:
from meerkat import parser

In [23]:
metadata, df = parser.csv_resource(m.csv_writer.path)

In [24]:
metadata

{'name': 'Grove Motor Driver',
 'title': None,
 'description': None,
 'format': None,
 'encoding': 'utf-8',
 'bytes': None,
 'hash': None,
 'schema': None,
 'sources': None,
 'licenses': None,
 'line_terminator': '\n',
 'quote_char': '"',
 'double_quote': True,
 'escape_char': '\\',
 'null_sequence': 'NA',
 'comment': '#',
 'skip_lines': 1,
 'path': '2020_01_07_07_44_33_data.csv',
 'device': {'name': 'Grove Motor Driver',
  'version_hw': '1.3',
  'version_sw': '1.0',
  'accuracy': None,
  'precision': '1 microstep',
  'bus': '<meerkat.i2c_pi.WrapI2C object at 0xb33f44f0>',
  'state': None,
  'active': None,
  'error': None,
  'dtype': None,
  'description': 'I2C DC and stepper motor controller',
  'urls': 'http://wiki.seeedstudio.com/Grove-I2C_Motor_Driver_V1.3/',
  'manufacturer': 'Seeed Studio',
  'calibration_date': None},
 'units': None,
 'dtypes': None,
 'accuracy': None,
 'precision': None,
 'time_format': 'std_time_ms',
 'strfmtime': '%Y-%m-%d %H:%M:%S.%f',
 'version': '0.1 Alph

In [25]:
df

,std_time_ms,description,freq,m1_speed,m2_speed,m_direction,mode,phase,steps,datetime64_ns
0,2020-01-07 07:44:33.109617,test_1b_init,31372,0,0,NaN,dc,2,0,2020-01-07 07:44:33.109617
1,2020-01-07 07:44:43.018238,test_2b_microsteps,31372,0,0,step_ccw,micro_step,2,-1001,2020-01-07 07:44:43.018238
2,2020-01-07 07:44:55.813782,test_3b_full_steps,31372,0,0,step_ccw,full_step,2,-100,2020-01-07 07:44:55.813782
3,2020-01-07 07:45:57.726786,test_5b_dc_cw,1,255,0,10,dc,2,-100,2020-01-07 07:45:57.726786
4,2020-01-07 07:45:59.732948,test_5e_dc_cw,1,0,0,10,dc,2,-100,2020-01-07 07:45:59.732948
5,2020-01-07 07:45:59.736628,test_5b_dc_ccw,1,255,0,5,dc,2,-100,2020-01-07 07:45:59.736628
6,2020-01-07 07:46:01.742367,test_5e_dc_ccw,1,0,0,5,dc,2,-100,2020-01-07 07:46:01.742367
7,2020-01-07 07:46:09.635181,test_6_0,1,0,0,5,dc,2,-100,2020-01-07 07:46:09.635181
8,2020-01-07 07:46:09.888703,test_6_25,1,25,0,5,dc,2,-100,2020-01-07 07:46:09.888703
9,2020-01-07 07:46:10.142647,test_6_50,1,50,0,5,dc,2,-100,2020-01-07 07:46:10.142647
